<a href="https://colab.research.google.com/github/ordevoir/Python/blob/main/15.7_%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D1%8B_-_%D0%94%D0%B5%D1%81%D0%BA%D1%80%D0%B8%D0%BF%D1%82%D0%BE%D1%80%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Дескриптор** (*descriptor*) – это объект, являющийся атрибутом класса и управляющий доступом к атрибутам экземпляров этого класса. Дескриптор реализует **протокол дескрипторов** (*descriptor protocol*), определяемый методами `__get__()`, `__set__()` и `__delete__()`. Дескрипторы лежат в основе работы `property`, `classmethod`, `staticmethod` и **связывания методов** (*method binding*).

Формально дескриптором считается объект, в классе которого реализован хотя бы один из методов `__get__`, `__set__` или `__delete__`. Если определён `__set__` и/или `__delete__`, объект является **дескриптором данных** (*data descriptor*); если определён только `__get__` – **дескриптором не-данных** (*non-data descriptor*). Эти типы различаются приоритетом при поиске атрибутов: data descriptor имеет приоритет над `instance.__dict__`, а non-data descriptor – нет.

Класс, в котором дескриптор определён как атрибут, называется **классом-владельцем** (*owner class*). Дескриптор присваивается атрибуту *класса* напрямую или применяется к методу как декоратор. При создании класса-владельца, у дескриптора вызывается метод `__set_name__()`, которому передается сам класс-владелец и имя атрибута, к которому привязан дескриптор.

## Рабочий пример

Пусть имеется класс для биологических клеток. В этом классе определены два поля объекта:
- `self.cell_type` для хранения типа клетки в виде строки;
- `self.diameter_um` для хранения диаметра клетки.

In [ ]:
class Cell:
    def __init__(self, cell_type, diameter_um):
        self.cell_type = cell_type
        self.diameter_um = diameter_um

Мы хотели бы, чтобы в `self.diameter_um` нельзя было бы установить не числовое значение, и чтобы число было положительным. Это можно сделать через дескриптор.

### Класс дескриптора

Определим класс для дескрипторов `PositiveNumber` для валидации положительных чисел. Этот класс не является непосредственно связанным с классом `Cell`. Мы создаем универсальный дескриптор, который можно создать в любом классе, если нам подходит протокол дескриптора. Протокол определяется определенными в классе `PositiveNumber` методами:

In [ ]:
class PositiveNumber:
    """Дескриптор данных для валидации положительных чисел.
    """
    def __get__(self, instance, owner):
        """Вызывается при чтении атрибута."""
        print(f"вызван метод __get__ дескриптора объекта {instance}, класса {owner}")
        if instance is None:
            return self  # доступ через класс, а не экземпляр
        return getattr(instance, self.storage_name, None)

    def __set__(self, instance, value):
        """Вызывается при записи атрибута."""
        print(f"вызван метод __set__ дескриптора объекта {instance}, и передано значение {value}")
        if not isinstance(value, (int, float)):
            raise TypeError(f'{self.name} должен быть числом')
        if value <= 0:
            raise ValueError(f'{self.name} должен быть положительным')
        setattr(instance, self.storage_name, value)

    def __set_name__(self, owner, name):
        """Вызывается при создании класса-владельца."""
        print(f"вызван метод __set_name__ дескриптора при создании дескриптора в классе {owner} с именем {name}")
        self.name = name                    # имя атрибута
        self.storage_name = f'__{name}'     # имя для хранения

### Создание дескриптора

Для того, чтобы использовать дескриптор в классе, необходимо создать экземпляр дескриптора и присвоить его атрибуту класса:

In [ ]:
class Cell:
    diameter_um = PositiveNumber()  # Дескриптор как атрибут КЛАССА
    def __init__(self, cell_type: str, diameter_um: float):
        self.cell_type = cell_type
        self.diameter_um = diameter_um  # вызовет __set__

вызван метод __set_name__ дескриптора при создании дескриптора в классе <class '__main__.Cell'> с именем diameter_um


Рассмотрим каждый метод подробнее:

### Метод `__set_name__()`

Метод `__set_name__(self, owner, name)` вызывается автоматически интерпретатором в момент создания класса-владельца, когда дескриптор присваивается атрибуту класса. Метод получает:

- `self` – сам объект-дескриптор;
- `owner` – класс-владелец, в котором дескриптор определён как атрибут;
- `name` – имя атрибута, которому присвоен дескриптор.

Данный дескриптор сохраняет имя поля объекта (`self.name`) для использования в сообщениях об ошибках, а также формирует `self.storage_name` – имя приватного поля экземпляра, в котором будет фактически храниться значение. Это важный паттерн: дескриптор не хранит данные в себе (иначе все экземпляры делили бы одно значение), а использует `__dict__` экземпляра.

>От того, что в `__set_name__()` определено поле `self.storage_name`, при создании объекта класса-владельца автоматически не возникает поле с именем, которое хранится в `self.storage_name`. Значение `self.storage_name` непосредственно используется методами `__get__()` и `__set__()`.

Когда интерпретатор выполняет определение класса `Cell`, происходит вызов метода дескриптора `diameter_um.__set_name__(Cell, 'diameter_um')`.

### Метод `__set__()`

Метод `__set__()` вызывается при записи значения в атрибут, управляемый дескриптором.

В нашем случае, метод `__set__()` дескриптора будет вызван при создании экземпляра класса `Cell`, так как в методе `__init__()` есть инструкция `self.diameter_um = diameter_um`. В этой инструкции идет обращение к имени `self.diameter_um`, которое уже определено в классе и является дескриптором данных. Так как дескриптор данных имеет приоритет выше атрибутов объекта, здесь не будет создано поле объекта `self.diameter_um`: вместо этого интерпретатор обратится к дескриптору. Справа от `self.diameter_um` имеется знак присваивания, поэтому будет вызван метод `__set__(self, instance, value)` дескриптора.

>`self.cell_type = cell_type` – в этой инструкции просто создается поле объекта `self.cell_type`, которому присваивается значение `cell_type` из аргумента.

Метод получит:

- `self` – объект-дескриптор;
- `instance` – экземпляр класса-владельца, атрибут которого изменяется;
- `value` – присваиваемое значение.

In [ ]:
cell = Cell("neuron", 30.0)

вызван метод __set__ дескриптора объекта <__main__.Cell object at 0x0000025C097C3800>, и передано значение 30.0


Что непосредственно делает метод `__set__()` в нашем случае? Вначале производится проверка, является ли значение `value` числом, далее проверяется, является ли число положительным. Если `value` не является числом, генерируется исключение `TypeError`, если `value` является числом, но не положительным, то генерируется исключение `ValueError`. Если `value` проходит валидацию, то значение присваивается полю объекта `self.__diameter_um`. За это отвечает инструкция `setattr(instance, self.storage_name, value)`. В поле дескриптора `self.storage_name` содержится имя `__diameter_um`. Посмотрим данные объекта `cell`, и убедимся, что в них действительно появлось имя `__diameter_um`:

In [ ]:
cell.__dict__

{'cell_type': 'neuron', '__diameter_um': 30.0}

Метод `__set__()` дескриптора будет вызван при любой попытке записи значения в атрибут `diameter_um`:

In [ ]:
cell.diameter_um = 15

вызван метод __set__ дескриптора объекта <__main__.Cell object at 0x0000025C097C3800>, и передано значение 15


Если значение не пройдет валидацию, возникнет исключение:

In [ ]:
# cell.diameter_um = "15"
# cell.diameter_um = -15

Важно иметь в виду, что создание поля объекта `__diameter_um` выполняется внутри класса `PositiveNumber`, а не внутри `Cell`. Поэтому, `__diameter_um` остается публичным полем, и вообще говоря, может быть изменен напрямую.

>Дело в том, что механизм искажения имён (*name mangling*) работает только на этапе компиляции класса, когда интерпретатор встречает `__name` в исходном коде класса. Поскольку `setattr(instance, '__diameter_um', value)` выполняется во время выполнения (*runtime*), искажение не применяется, и атрибут остаётся доступным как `instance.__diameter_um`. Более распространенная практика: использовать одинарное подчёркивание `_`.

In [ ]:
cell.__diameter_um = 14

In [ ]:
cell.__diameter_um

14

Таким образом имеем фактическое поле объекта `cell.__diameter_um`, с которым пользователь класса `Cell` не взаимодействует явно, и объект дескриптор `Cell.diameter_um`, отвечающий за чтение и запись значений поля `cell.__diameter_um`.

### Метод `__get__()`

Метод дескриптора `__get__()` вызывается при чтении атрибута, управляемого дескриптором. Метод принимает:

- `self` – объект-дескриптор;
- `instance` – экземпляр класса-владельца (или `None`, если доступ производится через класс);
- `owner` – класс-владелец.

In [ ]:
cell.diameter_um

вызван метод __get__ дескриптора объекта <__main__.Cell object at 0x0000025C097C3800>, класса <class '__main__.Cell'>


14

В нашей реализации, при вызове метода `__get__()` сначала производится проверка контекста вызова: если `instance is None`, значит, атрибут читается через класс, а не через экземпляр. В этом случае принято возвращать сам дескриптор – это позволяет инспектировать его:

In [ ]:
Cell.diameter_um

вызван метод __get__ дескриптора объекта None, класса <class '__main__.Cell'>


Если атрибут читается через объект, то `instance` будет ссылаться на объект, поэтому выполнится `getattr(instance, self.storage_name, None)` метод `__get__()` вернет результат.

В нашем примере, при  выполнении `cell.diameter_um` интерпретатор вызывает `Cell.diameter_um.__get__(cell, Cell)`.

In [ ]:
type(cell).__mro__

(__main__.Cell, object)

## Приоритет поиска атрибутов

При чтении атрибута `obj.attr` применяется **приоритет поиска атрибутов** (*attribute lookup order*):
1. Data descriptor.
1. Instance attribute.
1. Non-data descriptor.
1. Class attribute.

Если нигде не определен атрибут `attr`, вызывается метод `__getattr__()`, если он определен.

При записи атрибута `obj.attr = value` логика приоритета проще:
1. Data descriptor.
1. Instance attribute.

Таким образом, дескриптор данных выше по приоритету, чем атрибут объекта, поэтому, в инструкции `self.diameter_um = diameter_um` инициализатора, интерпретатор обратился к дескриптору данных `diameter_um`, вместо того, чтобы создать атрибут экземпляра, ведь в классе был определен дескриптор данных с именем `diameter_um` как атрибут класса.

Если бы `diameter_um` был просто атрибутом класса, или же был дескриптором не-данных, то по приоритету поиска интерпретатор создал бы атрибут объекта `self.diameter_um`. Мы не можем присвоить значение атрибуту класса через атрибут объекта.

## Встроенные дескрипторы

### Функции как дескрипторы

Любая функция является дескриптором не-данных: Функция является экземпляром класса `function`, у которого определен метод объекта `__get__()`:

In [ ]:
def f(a=None):
    print(f"f is called; object: {a}")

f.__get__

<method-wrapper '__get__' of function object at 0x0000025C09C59760>

При помощи метода `__get__()` можно привязать функцию к объекту, т.е. сделать его методом (*method binding*). Пусть имеется класс `Cell` и его объект `cell`:

In [ ]:
class Cell:
    def __init__(self, cell_type):
        self.cell_type = cell_type


cell = Cell("neuron")

Если передать в метод `__get__()` экземпляр класса и сам класс, то `__get__()` вернет метод:

In [ ]:
m = f.__get__(cell, Cell)
type(m)

method

На основе функции `f` мы получаем метод `m` объекта `cell` – функцию, связанную с объектом `cell`. Если вызвать этот метод, то первым аргументом он получит объект `cell`:

In [ ]:
m()

f is called; object: <__main__.Cell object at 0x0000025C097C2D80>


При определении метода в классе, создается просто функция:

In [ ]:
class Cell:
    def __init__(self, cell_type):
        self.cell_type = cell_type

    def describe(self):
        return f"Клетка: {self.cell_type}"

В данном примере создается функция `describe()` и мы можем увидеть её в словаре атрибутов класса:

In [ ]:
Cell.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Cell.__init__(self, cell_type)>,
              'describe': <function __main__.Cell.describe(self)>,
              '__dict__': <attribute '__dict__' of 'Cell' objects>,
              '__weakref__': <attribute '__weakref__' of 'Cell' objects>,
              '__doc__': None})

Это такой же атрибут, как и любой другой атрибут класса. Но так как у функции есть метод `__get__()`, этот атрибут является дескриптором не-данных. Так что теперь, если обратиться к этому атрибуту через экземпляр класса, будет вызван метод `__get__()` дескриптора `describe`, которому будет передан экземпляр (`instance`) и класс-владелец (`owner`):

```python
Cell.describe.__get__(cell, Cell)
```
или, если быть точнее:

```python
type(cell).__dict__['describe'].__get__(cell, type(cell))
```

Это выражение возвращает метод:

In [ ]:
cell = Cell("neuron")

m = cell.describe   # bound method
type(m)

method

Можно вызвать метод:

In [ ]:
m()

'Клетка: neuron'

Т.е. method binding происходит при обращении к атрибуту функции (`describe`) через объект.

Так как методы объекта, определенные в классе фактически являются дескрипторами не-данных, то поле объекта имеет приоритет над методом. Поэтому, если у экземпляра имеется поле с тем же именем, то при обращении к имени через объект, интерпретатор возьмёт именно поле объекта а не метод:

In [ ]:
class Cell:
    def __init__(self, cell_type):
        self.cell_type = cell_type
        self.describe = 10

    def describe(self):
        return f"Клетка: {self.cell_type}"

cell = Cell("neuron")
cell.describe

10

In [ ]:
Cell.describe

<function __main__.Cell.describe(self)>

Таким образом, метод объекта не является строго говоря атрибутом объекта – он является атрибутом класса. Поэтому, его нет в словаре `__dict__` объекта.

### Управление методами

Дескрипторы, модифицирующие поведение методов, применяются в виде декораторов.

Класс `staticmethod` определяет дескриптор не-данных, который отключает method binding.

In [ ]:
staticmethod.__dict__

mappingproxy({'__new__': <function staticmethod.__new__(*args, **kwargs)>,
              '__repr__': <slot wrapper '__repr__' of 'staticmethod' objects>,
              '__call__': <slot wrapper '__call__' of 'staticmethod' objects>,
              '__get__': <slot wrapper '__get__' of 'staticmethod' objects>,
              '__init__': <slot wrapper '__init__' of 'staticmethod' objects>,
              '__func__': <member '__func__' of 'staticmethod' objects>,
              '__wrapped__': <member '__wrapped__' of 'staticmethod' objects>,
              '__isabstractmethod__': <attribute '__isabstractmethod__' of 'staticmethod' objects>,
              '__dict__': <attribute '__dict__' of 'staticmethod' objects>,
              '__doc__': 'staticmethod(function) -> method\n\nConvert a function to be a static method.\n\nA static method does not receive an implicit first argument.\nTo declare a static method, use this idiom:\n\n     class C:\n         @staticmethod\n         def f(arg1, arg2, a

Класс `classmethod` определяет дескриптор не-данных, который возвращает связанный метод (bound method), где первым аргументом автоматически передаётся класс (а не экземпляр). Т.е. функция привязывается к классу, а не к экземпляру.

In [ ]:
classmethod.__dict__

mappingproxy({'__new__': <function classmethod.__new__(*args, **kwargs)>,
              '__repr__': <slot wrapper '__repr__' of 'classmethod' objects>,
              '__get__': <slot wrapper '__get__' of 'classmethod' objects>,
              '__init__': <slot wrapper '__init__' of 'classmethod' objects>,
              '__func__': <member '__func__' of 'classmethod' objects>,
              '__wrapped__': <member '__wrapped__' of 'classmethod' objects>,
              '__isabstractmethod__': <attribute '__isabstractmethod__' of 'classmethod' objects>,
              '__dict__': <attribute '__dict__' of 'classmethod' objects>,
              '__doc__': 'classmethod(function) -> method\n\nConvert a function to be a class method.\n\nA class method receives the class as implicit first argument,\njust like an instance method receives the instance.\nTo declare a class method, use this idiom:\n\n  class C:\n      @classmethod\n      def f(cls, arg1, arg2, argN):\n          ...\n\nIt can be called 

Класс `property` определяет дескриптор данных: в нем определены как метод `__get__()`, так и методы `__set__()` и `__delete__()`:

In [ ]:
property.__dict__

mappingproxy({'__new__': <function property.__new__(*args, **kwargs)>,
              '__getattribute__': <slot wrapper '__getattribute__' of 'property' objects>,
              '__get__': <slot wrapper '__get__' of 'property' objects>,
              '__set__': <slot wrapper '__set__' of 'property' objects>,
              '__delete__': <slot wrapper '__delete__' of 'property' objects>,
              '__init__': <slot wrapper '__init__' of 'property' objects>,
              'getter': <method 'getter' of 'property' objects>,
              'setter': <method 'setter' of 'property' objects>,
              'deleter': <method 'deleter' of 'property' objects>,
              '__set_name__': <method '__set_name__' of 'property' objects>,
              'fget': <member 'fget' of 'property' objects>,
              'fset': <member 'fset' of 'property' objects>,
              'fdel': <member 'fdel' of 'property' objects>,
              '__doc__': <member '__doc__' of 'property' objects>,
              

Сводная таблица встроенных дескрипторов

| Дескриптор | `__get__` | `__set__` | `__delete__` | Тип |
|------------|-----------|-----------|--------------|-----|
| `function` | ✅ | ❌ | ❌ | non-data |
| `property` | ✅ | ✅ | ✅ | data |
| `classmethod` | ✅ | ❌ | ❌ | non-data |
| `staticmethod` | ✅ | ❌ | ❌ | non-data |
| `member_descriptor` (slots) | ✅ | ✅ | ✅ | data |

---
